In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.pipeline import Pipeline
#from keras.wrappers.scikit_learn import KerasClassifier
from scikeras.wrappers import KerasClassifier, KerasRegressor
#from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
import pickle

In [2]:
data = pd.read_csv('Churn_Modelling.csv')

data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

one_hot_encoder_geography = OneHotEncoder(handle_unknown='ignore')
geo_encoded = one_hot_encoder_geography.fit_transform(data[['Geography']]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=one_hot_encoder_geography.get_feature_names_out(['Geography']))

data = pd.concat([data.drop(['Geography'], axis=1), geo_encoded_df], axis=1)

X = data.drop(['Exited'], axis=1)
y = data['Exited']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Save encoders and scaler for later use
with open('label_encoder_gender2.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender, file)

with open('one_hot_encoder_geography2.pkl', 'wb') as file:
    pickle.dump(one_hot_encoder_geography, file)
    
with open('scaler2.pkl', 'wb') as file:
    pickle.dump(scaler, file)



In [3]:
X_train.shape

(8000, 12)

In [4]:
# Define a function to create the model and try different parameters

def create_model(neurons=32, layers=1):
    model = Sequential()
    model.add(Dense(neurons, activation='relu', input_shape=(X_train.shape[1],)))
    
    for _ in range(layers-1):
        model.add(Dense(neurons, activation='relu'))
        
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [5]:
## Create a Keras Classifier

model = KerasClassifier(layers=1, neurons=32, build_fn=create_model, verbose=1)



In [6]:
# Define Grid Search parameters

param_grid = {
    'neurons': [16, 32, 64, 128],
    'layers': [1, 2],
    'epochs': [50, 100]
}

In [7]:
# Perform the grid search

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train)

# Print the optimal parameters

print(f'Best: {grid_result.best_score_} using {grid_result.best_params_}')


/opt/homebrew/anaconda3/lib/python3.12/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/opt/homebrew/anaconda3/lib/python3.12/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/opt/homebrew/anaconda3/lib/python3.12/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/opt/homebrew/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first lay

Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7848 - loss: 0.5407  
Epoch 2/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7001 - loss: 0.5827
Epoch 2/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7518 - loss: 0.5483
Epoch 2/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6174 - loss: 0.6355
Epoch 2/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.4535 - loss: 0.7818
Epoch 2/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.3333 - loss: 1.0644
167/167 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5565 - loss: 0.6777
Epoch 2/50
Epoch 2/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6444 - loss: 0.6356
167/167 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7905 - loss: 0.5068
Epoch 2/50
Epoch 2/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.6698 - loss: 0.5783
Epoch 2/50
167/16

/opt/homebrew/anaconda3/lib/python3.12/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/opt/homebrew/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8764 - loss: 0.3031
Epoch 48/50
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step   - accuracy: 0.8708 - loss: 0.3230
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8699 - loss: 0.3226
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8706 - loss: 0.3112
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 740us/step - accuracy: 0.8597 - loss: 0.3369
Epoch 47/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8620 - loss: 0.3179
Epoch 46/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8679 - loss: 0.3142
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 958us/step - accuracy: 0.8698 - loss: 0.3068
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8703 - loss: 0.3127
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8582 - loss: 0.3335 
Epoch 49/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8720 - loss: 0.30991
154/167 ━━━━━━━━━━━━━━━━━━━━ 0s 659us/step - accuracy: 0.8755 - loss: 0.3105Epoc

/opt/homebrew/anaconda3/lib/python3.12/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/opt/homebrew/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/opt/homebrew/anaconda3/lib/python3.12/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/opt/homebrew/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to 

...

  1/167 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - accuracy: 0.7812 - loss: 0.4328Epoch 1/100
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/stepep - accuracy: 0.8440 - loss: 0.4057
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8890 - loss: 0.2737
Epoch 99/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8231 - loss: 0.4039
Epoch 5/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8611 - loss: 0.3391
Epoch 11/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8312 - loss: 0.3992
 84/167 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8396 - loss: 0.3923Epoch 4/100
122/167 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8049 - loss: 0.4299Epoch 1/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 801us/step - accuracy: 0.8813 - loss: 0.2867
165/167 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8074 - loss: 0.4275Epoch 100/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8793 - loss: 0.2829
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8435 - loss: 0.3821
Epoch 5/100
1

/opt/homebrew/anaconda3/lib/python3.12/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/opt/homebrew/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/opt/homebrew/anaconda3/lib/python3.12/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/opt/homebrew/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to 

167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8463 - loss: 0.3793
Epoch 7/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8526 - loss: 0.3488  
Epoch 13/100
 98/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8529 - loss: 0.3563  Epoch 1/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8428 - loss: 0.3808
Epoch 5/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8448 - loss: 0.365833
Epoch 4/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8634 - loss: 0.3449
167/167 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7834 - loss: 0.4933
Epoch 2/100
Epoch 7/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 992us/step - accuracy: 0.8580 - loss: 0.3442
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8549 - loss: 0.3532  
Epoch 7/100
Epoch 12/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 923us/step - accuracy: 0.8564 - loss: 0.3444
Epoch 8/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8641 - loss: 0.3265
Epoch 14/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 

/opt/homebrew/anaconda3/lib/python3.12/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/opt/homebrew/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 353us/step - accuracy: 0.7393 - loss: 0.5435
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 308us/step - accuracy: 0.8061 - loss: 0.4521
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 310us/step - accuracy: 0.8232 - loss: 0.4206
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 314us/step - accuracy: 0.8238 - loss: 0.4186
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 303us/step - accuracy: 0.8367 - loss: 0.4046
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 303us/step - accuracy: 0.8427 - loss: 0.3835
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 325us/step - accuracy: 0.8426 - loss: 0.3738
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 486us/step - accuracy: 0.8427 - loss: 0.3792
Epoch 9/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 312us/step - accuracy: 0.8434 - loss: 0.3713
Epoch 10/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 309us/step - accuracy: 0.8530 - loss: 0.3617
Epoch 11/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 311us/step - accuracy: 0.8471 - loss: 0.3647
Epoch 12/100
250/250 ━━━━━━━━━━

In [16]:
# Run the model with the optimal parameters

model = KerasClassifier(layers=2, neurons=128, epochs=100, build_fn=create_model, verbose=1)

model.fit(X_train, y_train)

Epoch 1/100


/opt/homebrew/anaconda3/lib/python3.12/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/opt/homebrew/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 445us/step - accuracy: 0.7955 - loss: 0.4673
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 406us/step - accuracy: 0.8531 - loss: 0.3614
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 383us/step - accuracy: 0.8556 - loss: 0.3437
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 390us/step - accuracy: 0.8582 - loss: 0.3395
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 397us/step - accuracy: 0.8640 - loss: 0.3320
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 413us/step - accuracy: 0.8596 - loss: 0.3401
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 422us/step - accuracy: 0.8699 - loss: 0.3168
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 407us/step - accuracy: 0.8757 - loss: 0.3070
Epoch 9/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 456us/step - accuracy: 0.8689 - loss: 0.3120
Epoch 10/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 406us/step - accuracy: 0.8696 - loss: 0.3167
Epoch 11/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 409us/step - accuracy: 0.8781 - loss: 0.3008
Epoch 12/100
250/250 ━━━━━━━━━━

KerasClassifier(
	model=None
	build_fn=<function create_model at 0x321ebe5c0>
	warm_start=False
	random_state=None
	optimizer=rmsprop
	loss=None
	metrics=None
	batch_size=None
	validation_batch_size=None
	verbose=1
	callbacks=None
	validation_split=0.0
	shuffle=True
	run_eagerly=False
	epochs=100
	layers=2
	neurons=128
	class_weight=None
)